<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Peter Luschny - January 2026

#Composable, sparse, and rigid numbers
<h3 style="color:#CD5C5C;background:white; line-height: 100%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-bottom: 1em; margin-top: -0.5em;">

<center>
<img src="ComposableNumberTree.png" width="680" height="400">
</center>

 <pre>
A001477  (integers)
├── {0, 1}
├── A000040  (primes)
│   └── A392439  (composable-preceded primes)
└── A002808  (composites)
    ├── A392440  (composable)
    │   ├── A392438  (2-composable)
    │   └── A392437  (3-composable)
    └── A392500  (noncomposable)
        ├── A392498  (sparse)
        │   ├── A392493  (2-sparse)
        │   └── A392494  (3-sparse)
        └── A392499  (rigid)
            ├── A392496  (2-rigid)
            └── A392495  (3-rigid)
</pre>

## Mathematical concepts

**Notation:**

    Let n be a composite number and p denote its smallest prime factor.

    Let L = n/p be the largest proper divisor of n.

    Let D_{p}(n) be the set of admissible divisors of n defined as
 
    D_{p}(n) = {d in N : d | n and 1 < d < L }.

**Composable number:**
- A composable number is a composite number n for which there exists a subset of its admissible divisors D_{p}(n) that sums to L.
- We call these numbers 'composable numbers' because their admissible divisors can be combined in such a way that they sum up to the largest proper divisor of n.
- Examples: 12, 18, 24, 30, 36, 40, 42, 48, 54, 56, 60, ...
 
**Rigid number:**
- A rigid number is a noncomposable number with Sum(D_{p}(n)) >= L, but no subset sums to L.
- We call these numbers 'rigid numbers' to indicate that, although there are sufficiently many admissible divisors D_{p}(n) in total, their arithmetic structure is too rigid to allow to assemble the largest proper divisor of n. 
- Examples: 20, 45, 70, 88, 104, 105, 165, 175, 189, 195, ...
 
**Sparse number:**
- A sparse number is a noncomposable number with Sum(D_{p}(n)) < L.
- We call these numbers 'sparse numbers' because their admissible divisors are too few in total to reach the largest proper divisor of n.
- Examples: 4, 6, 8, 9, 10, 14, 15, 16, 21, 22, 25, 26, 27, ...
 

## Some comparison with classical classification

| Feature                   | Classical<br>(deficient / perfect / abundant) | Structural<br>(sparse / rigid / composable) |
| ------------------------- | ------------------------------------------ | ---------------------------------------- |
| **Domain**                | All integers n > 0                         | Composite numbers                        |
| **Smallest prime factor** | No special role                            | Central: p = smallest prime factor       |
| **Target value of sum**   | The number itself: n                       | Largest proper divisor: n/p              |
| **Admissible divisors**   | All proper divisors (including 1)          | Divisors d with 1 < d < n/p              |
| **Role of primes and 1**  | Included as deficient                      | Trivially excluded                       |
| **Decision criterion**    | Compare Sum_{d<n} d with n                 | Can some *subset* sum exactly to n/p?    |
| **Nature of test**        | Global sum comparison                      | Existence of a subset sum                |

Although the structural notions are formally defined for all integers n>1, 
prime numbers enter only as trivial noncomposable cases, since no admissible
divisors are available. This fact makes the non-trivial divisors a better 
filter than the proper divisors and leads to conceptually sharper defined classes.

**Computational Complexity:** Checking if a number is deficient, perfect, or abundant
relies on calculating the sum of proper divisors (the aliquot sum), meaning their 
complexity is determined by the divisor summation which is with an optimized divisor
search O(sqrt(n)). On the other hand, checking if a number is sparse, rigid, or 
composable is an NP-complete problem (the Subset Sum Problem) applied to the divisor set.

## Implementations

In [12]:
from math import isqrt
from functools import cache

In [13]:
def nontrivial_divisors(n: int) -> list[int]:
    """ Compute nontrivial divisors, in sorted order """
    if n == 0: return [0] # ok for our purposes
    divs = set()
    for d in range(2, isqrt(n) + 1):
        q, r = divmod(n, d)
        if r == 0:
            divs.add(d)
            divs.add(q)
    return sorted(divs)

In [14]:
def subset_sum_exists(nums: list[int], target: int) -> bool:
    """ Return whether some subset of nums sums to target """
    if sum(nums) < target:
        return False
    bits = 1  # bit i means sum i reachable
    for x in nums:
        bits |= bits << x
        if bits >> target & 1:
            return True
    return False

In [15]:
def is_composable(n: int) -> int:
    """Return the smallest prime divisor if n is composable else 0"""
    if n < 4:
        return 0  # False
    divs = nontrivial_divisors(n)
    if not divs:
        return 0  # False, prime

    D = divs[:-1]
    L = divs[-1]
    # Cheap eliminations, not part of definition
    if sum(D) < L:
        return 0  # False, sparse
    if len(divs) == 2:
        return 0  # False, semiprime

    b = divs[0] if subset_sum_exists(D, L) else 0
    return b

In [16]:
def is_sparse(n: int) -> int:
    """Return the smallest prime divisor if n is sparse else 0"""
    if n < 4:
        return 0  # False
    divs = nontrivial_divisors(n)
    if not divs:
        return 0  # False, n prime
    # divs[-1] = n/spf(n) is the largest nontrivial divisor

    b = divs[0] if sum(divs[:-1]) < divs[-1] else 0
    return b

In [17]:
def is_rigid(n: int) -> int:
    """Return the smallest prime divisor if n is rigid else 0"""
    if n < 4:
        return 0  # False
    divs = nontrivial_divisors(n)
    if not divs:
        return 0  # False, prime

    D = divs[:-1]
    L = divs[-1]
    S = sum(D)
    if S < L:
        return 0  # False

    b = 0 if subset_sum_exists(D, L) else divs[0]
    return b

In [18]:
def is_noncomposable(n: int) -> bool:
    return bool(is_sparse(n)) or bool(is_rigid(n))

In [19]:
from sympy import sieve

def composable_preceded_primes(upto: int) -> list[int]:
    primes = []
    for p in sieve.primerange(2, upto + 1):
        if is_composable(p - 1):
            primes.append(p)
    return primes

## Minimal terms

In [20]:
@cache
def smallest_prime_divisor(n: int) -> int:
    if n % 2 == 0:
        return 2
    for d in range(3, isqrt(n) + 1, 2):
        if n % d == 0:
            return d
    return n  # n is prime

In [21]:
def first_p_composable(p: int, limit: int = 10**12) -> int | None:
    # Optimization: Step by p if p=2, else step by 2p to skip even numbers
    step = p if p == 2 else 2 * p
    for n in range(p*p, limit + 1, step):
        if smallest_prime_divisor(n) != p: continue
        if is_composable(n):
            return n
    return None

In [22]:
def first_p_rigid(p: int, limit: int = 10**12) -> int | None:
    for n in range(p*p, limit+1, p):
        if smallest_prime_divisor(n) != p: continue
        if is_rigid(n):
            return n
    return None

In [23]:
def first_p_sparse(p: int, limit: int = 1) -> int:
    return p*p

### p-class-based

In [24]:
def is_p_composable(n: int, p: int) -> bool:
    return smallest_prime_divisor(n) == p and bool(is_composable(n))

In [25]:
def composable_list(p: int, limit: int) -> list[int]:
    return [n for n in range(p*p, limit+1, p) if is_p_composable(n, p)]

In [26]:
def is_p_rigid(n: int, p: int) -> bool:
    return smallest_prime_divisor(n) == p and bool(is_rigid(n))

In [27]:
def rigid_list(p: int, limit: int) -> list[int]:
    return [n for n in range(p*p, limit+1, p) if is_p_rigid(n, p)]

In [28]:
def is_p_sparse(n: int, p: int) -> bool:
    return smallest_prime_divisor(n) == p and bool(is_sparse(n))

In [29]:
def sparse_list(p: int, limit: int) -> list[int]:
    return [n for n in range(p*p, limit+1, p) if is_p_sparse(n, p)]

## Reducibility

In [30]:
from functools import cache

@cache
def is_sum_irreducible(n: int) -> bool:
    """n is sum_irreducible if no subset of its nontrivial divisors can sum to n."""
    divs = nontrivial_divisors(n)
    return not subset_sum_exists(divs, n)

In [31]:
def is_solid(n: int) -> bool:
    """n is solid if a subset of its sum_irreducible divisors sums to n."""
    divs = nontrivial_divisors(n)
    # Filter for components that are sum_irreducible
    irr_divs = [d for d in divs if is_sum_irreducible(d)]
    return subset_sum_exists(irr_divs, n)

### One-pass classifier

In [32]:
from math import isqrt

def classify_number(n: int) -> str:
    """
    Classifies n >= 0 into exactly one of:
    'idempotent', 'prime', 'sparse', 'rigid', or 'composable'. 
    """
    if n < 2:
        return "idempotent"

    # smallest prime divisor
    if n % 2 == 0:
        p = 2
    else:
        p = None
        for d in range(3, isqrt(n) + 1, 2):
            if n % d == 0:
                p = d
                break
        if p is None:
            return "prime"

    if p == n:
        return "prime"

    # compute L and D_p(n)
    L = n // p
    D = []
    for d in range(2, isqrt(n) + 1):
        if n % d == 0:
            q = n // d
            if d < L:
                D.append(d)
            if q != d and q < L:
                D.append(q)

    # sparse test (cheap, do first)
    total = sum(D)
    if total < L:
        return "sparse"

    # subset-sum test (bitset)
    bits = 1
    for x in D:
        bits |= bits << x
        if (bits >> L) & 1:
            return "composable"

    return "rigid"

In [33]:
for n in range(21): 
    print(f"{n:>3} | {classify_number(n)}")

  0 | idempotent
  1 | idempotent
  2 | prime
  3 | prime
  4 | sparse
  5 | prime
  6 | sparse
  7 | prime
  8 | sparse
  9 | sparse
 10 | sparse
 11 | prime
 12 | composable
 13 | prime
 14 | sparse
 15 | sparse
 16 | sparse
 17 | prime
 18 | composable
 19 | prime
 20 | rigid


<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">

# Math terms → OEIS sequences

In [34]:
TEST = True

## Composable numbers

In [35]:
def A392440_list(upto: int) -> list[int]:
    """
    Composable numbers 
    """
    return [n for n in range(2, upto + 1) if is_composable(n)]


if TEST: 
    print(A392440_list(252))
    # [12, 18, 24, 30, 36, 40, 42, 48, 54, 56, 60, 66, 72, ...]

[12, 18, 24, 30, 36, 40, 42, 48, 54, 56, 60, 66, 72, 78, 80, 84, 90, 96, 100, 102, 108, 112, 114, 120, 126, 132, 135, 138, 140, 144, 150, 156, 160, 162, 168, 174, 176, 180, 186, 192, 196, 198, 200, 204, 208, 210, 216, 220, 222, 224, 225, 228, 234, 240, 246, 252]


In [36]:
def A392440_p(p: int, upto: int) -> list[int]:
    """
    Classified composable
    """
    return composable_list(p, upto + 1)


if TEST: 
    print(A392440_p(2, 50))
    print(A392440_p(3, 500))
    print(A392440_p(5, 4025))
    print(A392440_p(7, 20111))
    print(A392440_p(11, 62491))
    # [12, 18, 24, 30, 36, 40, 42, 48]
    # [135, 225, 315, 345, 405, 495]
    # [1645, 1925, 2275, 3185, 4025]
    # [14651, 17017, 19019, 20111]
    # [8041, 23881, 46189, 62491]

[12, 18, 24, 30, 36, 40, 42, 48]
[135, 225, 315, 345, 405, 495]
[1645, 1925, 2275, 3185, 4025]
[14651, 17017, 19019, 20111]
[8041, 23881, 46189, 62491]


In [37]:
def A392438_list(upto: int) -> list[int]:
    """
    2-composable numbers
    """
    return composable_list(2, upto + 1)


if TEST: 
    print(A392438_list(260))
    # [12, 18, 24, 30, 36, 40, 42, 48, 54, 56, 60, 66, 72, ...]

[12, 18, 24, 30, 36, 40, 42, 48, 54, 56, 60, 66, 72, 78, 80, 84, 90, 96, 100, 102, 108, 112, 114, 120, 126, 132, 138, 140, 144, 150, 156, 160, 162, 168, 174, 176, 180, 186, 192, 196, 198, 200, 204, 208, 210, 216, 220, 222, 224, 228, 234, 240, 246, 252, 258, 260]


In [38]:
def A392437_list(upto: int) -> list[int]:
    """
    3-composable numbers
    """
    return composable_list(3, upto + 1)


if TEST: 
    print(A392437_list(2583))
    # [135, 225, 315, 345, 405, 495, 525, 585, 675, 765, ...]

[135, 225, 315, 345, 405, 495, 525, 585, 675, 765, 855, 945, 975, 1035, 1125, 1155, 1197, 1215, 1305, 1365, 1395, 1425, 1485, 1575, 1617, 1665, 1725, 1755, 1785, 1845, 1935, 1953, 1995, 2025, 2079, 2115, 2145, 2205, 2295, 2331, 2385, 2415, 2457, 2475, 2565, 2583]


## Sparse numbers

In [39]:
def A392498_list(upto: int) -> list[int]:
    """
    Sparse numbers
    """
    return [n for n in range(2, upto + 1) if is_sparse(n)]


if TEST: 
    print(A392498_list(125))
    # [4, 6, 8, 9, 10, 14, 15, 16, 21, 22, 25, 26, 27, 28, 32, ...]

[4, 6, 8, 9, 10, 14, 15, 16, 21, 22, 25, 26, 27, 28, 32, 33, 34, 35, 38, 39, 44, 46, 49, 50, 51, 52, 55, 57, 58, 62, 63, 64, 65, 68, 69, 74, 75, 76, 77, 81, 82, 85, 86, 87, 91, 92, 93, 94, 95, 98, 99, 106, 110, 111, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125]


In [40]:
def A392498_p(p: int, upto: int) -> list[int]:
    """
    Classified sparse
    """
    return sparse_list(p, upto + 1)


if TEST: 
    print(A392498_p(2, 20))
    print(A392498_p(3, 40))
    print(A392498_p(5, 100))
    print(A392498_p(7, 200))
    print(A392498_p(11, 320))
    # [4, 6, 8, 10, 14, 16]
    # [9, 15, 21, 27, 33, 39]
    # [25, 35, 55, 65, 85, 95]
    # [49, 77, 91, 119, 133, 161]
    # [121, 143, 187, 209, 253, 319]

[4, 6, 8, 10, 14, 16]
[9, 15, 21, 27, 33, 39]
[25, 35, 55, 65, 85, 95]
[49, 77, 91, 119, 133, 161]
[121, 143, 187, 209, 253, 319]


In [41]:
def A392493_list(upto: int) -> list[int]:
    """
    2-sparse numbers
    """
    return sparse_list(2, upto + 1)


if TEST: 
    print(A392493_list(172))
    # [4, 6, 8, 10, 14, 16, 22, 26, 28, 32, 34, ...]

[4, 6, 8, 10, 14, 16, 22, 26, 28, 32, 34, 38, 44, 46, 50, 52, 58, 62, 64, 68, 74, 76, 82, 86, 92, 94, 98, 106, 110, 116, 118, 122, 124, 128, 130, 134, 136, 142, 146, 148, 152, 154, 158, 164, 166, 170, 172]


In [42]:
def A392494_list(upto: int) -> list[int]:
    """
    3-sparse numbers
    """
    return sparse_list(3, upto + 1)


if TEST: 
    print(A392494_list(411))
    # [9, 15, 21, 27, 33, 39, 51, 57, 63, 69, 75, 81, ...]

[9, 15, 21, 27, 33, 39, 51, 57, 63, 69, 75, 81, 87, 93, 99, 111, 117, 123, 129, 141, 147, 153, 159, 171, 177, 183, 201, 207, 213, 219, 231, 237, 243, 249, 261, 267, 273, 279, 291, 297, 303, 309, 321, 327, 333, 339, 351, 357, 363, 369, 375, 381, 387, 393, 399, 411]


## Rigid numbers

In [43]:
def A392499_list(upto: int) -> list[int]:
    """
    Rigid numbers
    """
    return [n for n in range(2, upto + 1) if is_rigid(n)]


if TEST: 
    print(A392499_list(1504))
    # [20, 45, 70, 88, 104, 105, 165, 175, 189, 195, ...]

[20, 45, 70, 88, 104, 105, 165, 175, 189, 195, 255, 272, 285, 304, 350, 368, 385, 441, 455, 464, 567, 572, 595, 650, 665, 693, 715, 735, 805, 819, 825, 836, 875, 1001, 1015, 1071, 1085, 1184, 1225, 1275, 1287, 1295, 1309, 1312, 1323, 1376, 1435, 1449, 1463, 1504]


In [44]:
def A392499_p(p: int, upto: int) -> list[int]:
    """
    Classified rigid
    """
    return rigid_list(p, upto + 1)


if TEST: 
    print(A392499_p(2, 305))
    print(A392499_p(3, 255))
    print(A392499_p(5, 715))
    print(A392499_p(7, 1729))
    print(A392499_p(11, 3553))
    # [20, 70, 88, 104, 272, 304]
    # [45, 105, 165, 189, 195, 255]
    # [175, 385, 455, 595, 665, 715]
    # [1001, 1309, 1463, 1547, 1729]
    # [1573, 2431, 2717, 3289, 3553]

[20, 70, 88, 104, 272, 304]
[45, 105, 165, 189, 195, 255]
[175, 385, 455, 595, 665, 715]
[1001, 1309, 1463, 1547, 1729]
[1573, 2431, 2717, 3289, 3553]


In [45]:
def A392496_list(upto: int) -> list[int]:
    """
    2-rigid numbers
    """
    return rigid_list(2, upto + 1)


if TEST: 
    print(A392496_list(9730))
    # [20, 70, 88, 104, 272, 304, 350, 368, 464, 572, ...]

[20, 70, 88, 104, 272, 304, 350, 368, 464, 572, 650, 836, 1184, 1312, 1376, 1504, 1696, 1888, 1952, 3770, 4030, 4288, 4544, 4672, 5056, 5312, 5696, 5704, 5810, 5830, 6208, 6464, 6592, 6790, 6808, 6848, 6976, 7144, 7192, 7232, 7630, 7910, 7912, 8024, 8056, 9272, 9590, 9730]


In [46]:
def A392495_list(upto: int) -> list[int]:
    """
    3-rigid numbers
    """
    return rigid_list(3, upto + 1)


if TEST: 
    print(A392495_list(5607))
    # [45, 105, 165, 189, 195, 255, 285, 441, 567, ...]

[45, 105, 165, 189, 195, 255, 285, 441, 567, 693, 735, 819, 825, 1071, 1275, 1287, 1323, 1449, 1683, 1701, 1815, 1827, 1911, 2175, 2325, 2535, 2541, 2709, 2775, 3075, 3087, 3339, 3525, 3717, 3843, 3975, 4221, 4335, 4473, 4575, 5025, 5103, 5229, 5325, 5475, 5607]


## Noncomposable numbers

In [47]:
def A392500_list(upto: int) -> list[int]:
    """
    noncomposable numbers
    """
    return [n for n in range(2, upto + 1) if is_noncomposable(n)]

if TEST: 
    print(A392500_list(118))
    # [4, 6, 8, 9, 10, 14, 15, 16, 20, 21, 22, 25, 26, 27, ...]

[4, 6, 8, 9, 10, 14, 15, 16, 20, 21, 22, 25, 26, 27, 28, 32, 33, 34, 35, 38, 39, 44, 45, 46, 49, 50, 51, 52, 55, 57, 58, 62, 63, 64, 65, 68, 69, 70, 74, 75, 76, 77, 81, 82, 85, 86, 87, 88, 91, 92, 93, 94, 95, 98, 99, 104, 105, 106, 110, 111, 115, 116, 117, 118]


## Sum-irreducible numbers

In [48]:
def A392650_list(upto: int) -> list[int]:
    return [n for n in range(1, upto + 1) if is_sum_irreducible(n)]

if TEST: 
    print(A392650_list(100))
    # [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 19, ...]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 31, 32, 33, 34, 35, 37, 38, 39, 41, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 55, 57, 58, 59, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 79, 81, 82, 83, 85, 86, 87, 88, 89, 91, 92, 93, 94, 95, 97, 98, 99]


## Solid numbers

In [49]:
def A392649_list(upto: int) -> list[int]:
    """
    Solid numbers
    """
    return [n for n in range(2, upto + 1) if is_solid(n)]

if TEST:
    print(A392649_list(570))
    # [12, 18, 30, 40, 42, 56, 60, 66, 78, 90, 100, ...]

[12, 18, 30, 40, 42, 56, 60, 66, 78, 90, 100, 102, 114, 138, 140, 150, 174, 176, 186, 196, 208, 210, 220, 222, 246, 258, 260, 282, 294, 308, 318, 330, 340, 354, 364, 366, 380, 390, 402, 426, 438, 450, 460, 462, 474, 476, 490, 498, 510, 532, 534, 544, 546, 550, 570]


## Primes preceded by a composable number

In [50]:
def A392439_list(limit: int) -> list[int]:
    """
    composable-preceded primes
    """
    return composable_preceded_primes(limit + 1)


if TEST: 
    print(A392439_list(521))
    # [13, 19, 31, 37, 41, 43, 61, 67, 73, 79, 97, 101, ...]


[13, 19, 31, 37, 41, 43, 61, 67, 73, 79, 97, 101, 103, 109, 113, 127, 139, 151, 157, 163, 181, 193, 197, 199, 211, 223, 229, 241, 271, 277, 281, 283, 307, 313, 331, 337, 349, 353, 367, 373, 379, 397, 401, 409, 421, 433, 439, 449, 457, 461, 463, 487, 491, 499, 521]


## Smallest term in p-class

In [51]:
from sympy import sieve

In [52]:
def A392492_list(upto: int) -> list[int | None]:
    """
    Smallest p-composable number
    """
    return [first_p_composable(p) for p in sieve.primerange(2, upto + 1)]


if TEST:  # this takes some time!
    print(A392492_list(36))
    # [12, 135, 1645, 14651, 8041, 175253, 115957, 695267, 2128351, ...]

[12, 135, 1645, 14651, 8041, 175253, 115957, 695267, 2128351, 1590331, 5024263]


In [53]:
def A392497_list(upto: int) -> list[int | None]:
    """
    Smallest p-rigid number
    """
    return [first_p_rigid(p) for p in sieve.primerange(2, upto + 1)]


if TEST: 
    print(A392497_list(150))
    # [20, 45, 175, 1001, 1573, 4199, 5491, 12673, 20677, 26071, ...]

[20, 45, 175, 1001, 1573, 4199, 5491, 12673, 20677, 26071, 47027, 65231, 72283, 107113, 146969, 190747, 212341, 290177, 347261, 367993, 478661, 583573, 716539, 871933, 1009091, 1050703, 1201289, 1247941, 1564259, 1879981, 2279269, 2494633, 2608891, 3127361, 3352351]


## Completeness tests and distribution

In [54]:
def test_p_class_completeness(p: int, limit: int) -> None:
    """
    Verifies that for a prime p, every composite n with spf(n) = p
    is either p-sparse, p-rigid, or p-composable.
    """
    # Gather all composites in the p-class
    # (multiples of p where p is the smallest prime divisor)
    p_class_composites = []
    step = p if p == 2 else 2 * p
    start = p + step

    for n in range(start, limit + 1, step):
        if smallest_prime_divisor(n) == p:
            p_class_composites.append(n)

    # Get the lists 
    com = composable_list(p, limit)
    rig = rigid_list(p, limit)
    spa = sparse_list(p, limit )
    
    # Calculate the union
    union_count = len(com) + len(rig) + len(spa)
    total_expected = len(p_class_composites)

    # Reporting
    print(f"--- Completeness test for p={p} (up to {limit}) ---")
    print(f"Total composites in p-class: {total_expected}")
    print(f"  - sparse:     {len(spa):>6}")
    print(f"  - rigid:      {len(rig):>6}")
    print(f"  - composable: {len(com):>6}")
    print(f"Total categorized:           {union_count}")

    is_complete = union_count == total_expected
    print(f"p-completeness verified: {is_complete} <- **** \n")

In [55]:
for p in [2, 3, 5, 7, 11, 13]:
    test_p_class_completeness(p, 10**5)

--- Completeness test for p=2 (up to 100000) ---
Total composites in p-class: 49999
  - sparse:      25414
  - rigid:         304
  - composable:  24281
Total categorized:           49999
p-completeness verified: True <- **** 

--- Completeness test for p=3 (up to 100000) ---
Total composites in p-class: 16666
  - sparse:      13882
  - rigid:         541
  - composable:   2243
Total categorized:           16666
p-completeness verified: True <- **** 

--- Completeness test for p=5 (up to 100000) ---
Total composites in p-class: 6666
  - sparse:       6024
  - rigid:         323
  - composable:    319
Total categorized:           6666
p-completeness verified: True <- **** 

--- Completeness test for p=7 (up to 100000) ---
Total composites in p-class: 3808
  - sparse:       3655
  - rigid:         102
  - composable:     51
Total categorized:           3808
p-completeness verified: True <- **** 

--- Completeness test for p=11 (up to 100000) ---
Total composites in p-class: 2077
  - spar

In [56]:
from sympy.functions.combinatorial.numbers import primepi

def distribution(upto: int) -> None:
    """
    Verifies that every number n >= 1 and n <= limit 
    is either composable, sparse, rigid, prime, or {1}.
    """
    # Get the lists 
    com = [n for n in range(upto + 1) if is_composable(n)]
    rig = [n for n in range(upto + 1) if is_rigid(n)]
    spa = [n for n in range(upto + 1) if is_sparse(n)]
    
    # Calculate the union
    union_count = len(com) + len(rig) + len(spa)
    prime_pi = int(primepi(upto))
    total = union_count + prime_pi + 1  # +1 for '1' (idempotent)

    # Reporting
    print(f"--- Completeness test up to {upto} ---")
    print(f"Total composites: {union_count}")
    print(f"  - sparse:     {len(spa):>6}")
    print(f"  - composable: {len(com):>6}")
    print(f"  - prime:      {prime_pi:>6}")
    print(f"  - rigid:      {len(rig):>6}")
    print(f"Total categorized:          {total}")

    is_complete = upto == total
    print(f"Completeness verified: {is_complete} <- **** \n")

A first insight into the distribution gives
<pre>
      up to       1000000 
    - composites:  921501
    - sparse:      635799
    - composable:  270712
    - prime:        78498
    - rigid:        14990
</pre>

In [57]:
distribution(10**5)

--- Completeness test up to 100000 ---
Total composites: 90407
  - sparse:      61822
  - composable:  26903
  - prime:        9592
  - rigid:        1682
Total categorized:          100000
Completeness verified: True <- **** 



<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">

### <b>Exercises and open problems</b>

#### Is every irreducible composable number also an abundant number?

In [58]:
def is_abundant(n: int) -> bool:
    return sum(nontrivial_divisors(n)) > n + 1

In [59]:
def is_solid_and_not_abundant(limit: int) -> int:
    for n in range(1, limit):
        if is_solid(n):
            if not is_abundant(n):
                return n # Found a spoiler!
    return 0

print(is_solid_and_not_abundant(100000))
# Result for limit 100000: 0 [Takes about 7 seconds.]

0


#### Implement is_semiperfect using our framework.

In [60]:
def is_semiperfect(n: int) -> bool:
    if n < 6 or (n % 2 != 0 and n < 945): return False
    proper_divs = [1] + nontrivial_divisors(n)
    return subset_sum_exists(proper_divs, n)

A = [n for n in range(201) if is_semiperfect(n)]
print(A)

[6, 12, 18, 20, 24, 28, 30, 36, 40, 42, 48, 54, 56, 60, 66, 72, 78, 80, 84, 88, 90, 96, 100, 102, 104, 108, 112, 114, 120, 126, 132, 138, 140, 144, 150, 156, 160, 162, 168, 174, 176, 180, 186, 192, 196, 198, 200]


#### What are semiperfect numbers that are not solid?

These are numbers that are the sum of a subset of their nontrivial divisors 1 < d < n, but where every 
such subset contains at least one divisor that is itself the sum of its own nontrivial divisors.

Equivalently these are composable numbers that cannot be formed exclusively from their sum-irreducible divisors.

In [61]:
def is_A(n: int) -> bool:
    return is_semiperfect(n) and not is_solid(n)
A = [n for n in range(1, 201) if is_A(n)]
print(A)

[6, 20, 24, 28, 36, 48, 54, 72, 80, 84, 88, 96, 104, 108, 112, 120, 126, 132, 144, 156, 160, 162, 168, 180, 192, 198, 200]


#### Is 2-rigidity equivalent to sum-irreducible and not nonabundant?

In [62]:
def is_nonabundant(n: int) -> bool:
    # sigma(n) <= 2n is equivalent to sum of proper divisors <= n
    return sum(nontrivial_divisors(n)) < n

def is_sum_irreducible_and_nonabundant(n: int) -> bool:
    return is_sum_irreducible(n) and not is_nonabundant(n)

In [63]:
upto = 1000
A = [n for n in range(4, upto + 1) if is_sum_irreducible_and_nonabundant(n)]
B = A392496_list(upto)  # A392496 are the 2-rigid numbers
assert(A == B)
print(A)
print(B)

[20, 70, 88, 104, 272, 304, 350, 368, 464, 572, 650, 836]
[20, 70, 88, 104, 272, 304, 350, 368, 464, 572, 650, 836]
